<a href="https://colab.research.google.com/github/saif-mahmud/hierarchical-attention-HAR/blob/master/Hiererchical_self_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
class PositionalEncoding(tf.keras.layers.Layer):

    def __init__(self, position, d_model, include_dropout=True, dropout_rate=0.2):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        self.pos_encoding = self.positional_encoding(position, d_model)
        self.include_dropout = include_dropout
        if include_dropout:
            self.dropout = tf.keras.layers.Dropout(rate=dropout_rate)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model)
        # apply sin to even index in the array
        sines = tf.math.sin(angle_rads[:, 0::2])
        # apply cos to odd index in the array
        cosines = tf.math.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, x):
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = x + self.pos_encoding[:, :tf.shape(x)[1], :]
        if self.include_dropout:
            x = self.dropout(x)
        return x

In [0]:
class LearnedPositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, num_positions, d_model, include_dropout=True, dropout_rate=0.2):
        super(LearnedPositionalEncoding, self).__init__()
        self.d_model = d_model
        self.pos_encoding = tf.keras.layers.Embedding(num_positions, d_model)
        self.include_dropout = include_dropout
        if include_dropout:
            self.dropout = tf.keras.layers.Dropout(rate=dropout_rate)

    def call(self, x):
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = x + self.pos_encoding(tf.expand_dims(tf.range(tf.shape(x)[1]), axis=0))
        if self.include_dropout:
            x = self.dropout(x)
        return x

In [0]:
def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead) 
    but it must be broadcastable for addition.

    Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
            to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
    output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights


class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model,use_bias=False)
        self.wk = tf.keras.layers.Dense(d_model,use_bias=True)
        self.wv = tf.keras.layers.Dense(d_model,use_bias=True)

        self.dense = tf.keras.layers.Dense(d_model)
        
    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention, 
                                        (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
            
        return output, attention_weights


def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
        tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])

In [0]:
class AggregateAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(AggregateAttention, self).__init__()
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.d_model = d_model
        self.query = self.add_weight("learned_query",
                                     shape=[1, 1, self.d_model],
                                     initializer=tf.keras.initializers.Orthogonal())  
    
    def call(self, v, k):
        batched_query = tf.tile(self.query, [tf.shape(v)[0],1,1])
        output, attention_weights = self.mha(v, k, batched_query, mask=None)
        output = tf.squeeze(output, axis=1)
        attention_weights = tf.squeeze(attention_weights, axis=2)
        return output, attention_weights

Testing the shapes

In [6]:
temp_agg_attn = AggregateAttention(d_model=32, num_heads=2)

y = tf.random.uniform((128, 10, 32))  # (batch_size, sequence_len, d_model)

out, attn = temp_agg_attn(y, k=y)

out.shape, attn.shape

(TensorShape([128, 32]), TensorShape([128, 2, 10]))

In [0]:
class AggregateAttentionBlock(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, expand_dim_factor=1, rate=0.1):
        super(AggregateAttentionBlock, self).__init__()
        self.aga = AggregateAttention(d_model*expand_dim_factor, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model*expand_dim_factor, dff*expand_dim_factor)

        self.layernorm = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training):
        attn_output, attn_scores = self.aga(x, x)
        attn_output = self.dropout1(attn_output, training=training)
        
        ffn_output = self.ffn(attn_output)  # (batch_size, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out = self.layernorm(ffn_output + attn_output)  # (batch_size, d_model)
        return out, attn_scores

In [8]:
temp_agg_attnblock = AggregateAttentionBlock(d_model=32, num_heads=4, dff=128)

y = tf.random.uniform((16, 10, 32))  # (batch_size, sequence_len, d_model)

block_out, attn = temp_agg_attnblock(y, True)

block_out.shape, attn.shape

(TensorShape([16, 32]), TensorShape([16, 4, 10]))

In [0]:
class SelfAttentionBlock(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(SelfAttentionBlock, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    
  def call(self, x, training, mask=None):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)
    
    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)
    
    return out2

In [0]:
class EncoderBlock(tf.keras.layers.Layer):
    def __init__(self, n_timesteps, d_model, num_heads, dff, num_sa_blocks=1, add_pe=True, pe_type = 'learned', expand_dim_factor=1, dropout_rate=0.1):
        super(EncoderBlock, self).__init__()
        self.add_pe = add_pe
        if self.add_pe:
            if pe_type == 'fixed': 
                self.pe = PositionalEncoding(n_timesteps, d_model, dropout_rate=dropout_rate)
            else:
                self.pe = LearnedPositionalEncoding(n_timesteps, d_model, dropout_rate=dropout_rate)
        self.pe = PositionalEncoding(n_timesteps, d_model)
        self.self_attn_blocks = tf.keras.Sequential(
            [SelfAttentionBlock(d_model, num_heads, dff, dropout_rate) for _ in range(num_sa_blocks)]
            )
        self.attn_aggr = AggregateAttentionBlock(d_model, num_heads, dff, expand_dim_factor, dropout_rate)

    def call(self, x):
        if self.add_pe:
            x = self.pe(x)
        x = self.self_attn_blocks(x)
        x, _  = self.attn_aggr(x)
        return x

    def compute_output_shape(x, input_shape):
        return tf.TensorShape([input_shape[0], input_shape[-1]])

In [11]:
temp_encoder = EncoderBlock(20, d_model=32, num_heads=4, dff=128, num_sa_blocks=2)

y = tf.random.uniform((128, 10, 32))  # (batch_size, sequence_len, d_model)

block_out = temp_encoder(y)

block_out.shape

TensorShape([128, 32])

In [0]:
class HSAEncoder(tf.keras.layers.Layer):
    def __init__(self, n_window, n_timesteps, d_model, num_heads, dff, expand_dim_factor=1, dropout_rate=0.1):
        super(HSAEncoder, self).__init__()
        self.n_window = n_window
        self.d_model = d_model
        self.n_timesteps = n_timesteps
        self.expand_dim_factor = expand_dim_factor
        self.window_encoder = EncoderBlock(n_timesteps, d_model, num_heads,dff,
                                           num_sa_blocks=2, add_pe=True,
                                           pe_type = 'fixed',
                                           expand_dim_factor = expand_dim_factor, 
                                           dropout_rate=dropout_rate)
        self.session_encoder = EncoderBlock(n_window, d_model*expand_dim_factor,
                                            num_heads,dff*expand_dim_factor,
                                            num_sa_blocks=1, add_pe=True,
                                            pe_type = 'learned',
                                            expand_dim_factor = expand_dim_factor,
                                            dropout_rate=dropout_rate)
     
        
    def call(self, x):
        batch_size = tf.shape(x)[0]
        x = tf.reshape(x, (-1, self.n_timesteps, self.d_model)) # all sessions in batch dim
        x = self.window_encoder(x)
        x = tf.reshape(x, (batch_size, -1, self.d_model*self.expand_dim_factor))
        x = self.session_encoder(x)
        return x


    def compute_output_shape(x, input_shape):
        return tf.TensorShape([input_shape, self.d_model*self.expand_dim_factor**2])

In [13]:
temp_hier_attn = HSAEncoder(n_window=30, n_timesteps=100, d_model=64, num_heads=4, dff=128)

y = tf.random.uniform((16, 5, 100, 64))  # (batch_size, no_of_window, sequence_len, d_model)

block_out = temp_hier_attn(y)

block_out.shape

TensorShape([16, 64])

In [0]:
class MLPClassifier(tf.keras.layers.Layer):
    def __init__(self, num_classes, hidden_layers = [256]):
        super(MLPClassifier, self).__init__()
        self.layers = tf.keras.Sequential(
            [tf.keras.layers.Dense(num_units, activation='relu') for num_units in hidden_layers]
        )
        self.final_layer = tf.keras.layers.Dense(num_classes, activation='softmax')
        self.dropout = tf.keras.layers.Dropout(0.3)

    def call(self, x):
        x = self.layers(x)
        x = self.dropout(x)
        return self.final_layer(x)

In [0]:
class HARModel(tf.keras.Model):
    def __init__(self, num_classes, n_window, n_timesteps, d_model, num_heads, dff,expand_dim_factor=1, dropout_rate=0.3):
        super(HARModel, self).__init__()
        self.data_transform = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(d_model, 1, activation='relu'))
        self.encoder = HSAEncoder(n_window, n_timesteps, d_model, num_heads, dff,expand_dim_factor, dropout_rate)
        self.classifier = MLPClassifier(num_classes)

    def call(self, x):
        x = self.data_transform(x)
        x = self.encoder(x)
        return self.classifier(x)